In [ ]:
!sudo apt install tree
!python3 -m pip install --upgrade --user ortools

%cd /content
# !rm -rf kplib
!git clone https://github.com/likr/kplib

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.symlink('/content/drive/MyDrive/Knapsack/', '/content/Knapsack')

In [7]:
from ortools.algorithms import pywrapknapsack_solver
from pathlib import Path
from random import choice
from time import time

In [ ]:
!tree kplib -d

In [ ]:
!tree kplib

In [ ]:
# # Pick test cases
# cases = Path('/content/kplib',).glob('[0-1]*/n*')
# cases = [a.glob('*/*.kp') for a in cases]
# cases = [choice(list(a)) for a in cases]
# cases.sort()
# with open('/content/Knapsack/cases.txt', 'w') as f:
#   for c in cases:
#     f.write(str(c) + '\n')

In [13]:
# Load test case directories
with open('/content/Knapsack/cases.txt', 'r') as f:
  cases = ''.join(f.readlines()).strip('\n').split('\n')
for c in cases:
  print(c)
print(len(cases))

/content/kplib/00Uncorrelated/n00050/R10000/s035.kp
/content/kplib/00Uncorrelated/n00100/R10000/s098.kp
/content/kplib/00Uncorrelated/n00200/R01000/s033.kp
/content/kplib/00Uncorrelated/n00500/R10000/s054.kp
/content/kplib/00Uncorrelated/n01000/R10000/s011.kp
/content/kplib/00Uncorrelated/n02000/R01000/s086.kp
/content/kplib/00Uncorrelated/n05000/R10000/s000.kp
/content/kplib/00Uncorrelated/n10000/R01000/s012.kp
/content/kplib/01WeaklyCorrelated/n00050/R10000/s083.kp
/content/kplib/01WeaklyCorrelated/n00100/R01000/s036.kp
/content/kplib/01WeaklyCorrelated/n00200/R10000/s074.kp
/content/kplib/01WeaklyCorrelated/n00500/R01000/s039.kp
/content/kplib/01WeaklyCorrelated/n01000/R10000/s073.kp
/content/kplib/01WeaklyCorrelated/n02000/R01000/s092.kp
/content/kplib/01WeaklyCorrelated/n05000/R01000/s086.kp
/content/kplib/01WeaklyCorrelated/n10000/R01000/s031.kp
/content/kplib/02StronglyCorrelated/n00050/R10000/s058.kp
/content/kplib/02StronglyCorrelated/n00100/R10000/s060.kp
/content/kplib/02Str

In [2]:
def read_case(dir):
  weights = []
  values = []
  capacities = -1

  with open(str(dir),'r') as f:
    f.readline()
    n = int(f.readline().strip())
    capacities = int(f.readline().strip())
    f.readline()

    for i in range(n):
      v, w = map(int, f.readline().strip().split(' '))
      values.append(v)
      weights.append(w)
      
  weights = [weights]
  capacities = [capacities]
  return values, weights, capacities

In [9]:
def solve(values, weights, capacities, time_limit_seconds):
  packed_items = []
  packed_weights = []
  total_weight = 0
  optimality = 'Uncertain'

  solver = pywrapknapsack_solver.KnapsackSolver(
      pywrapknapsack_solver.KnapsackSolver.
      KNAPSACK_MULTIDIMENSION_BRANCH_AND_BOUND_SOLVER, 'KnapsackExample')
  solver.set_time_limit(time_limit_seconds)
  solver.Init(values, weights, capacities)

  start = time()
  computed_value = solver.Solve()
  end = time()
  if (end - start <= time_limit_seconds - 0.5):
    optimality = 'Optimal'
    
  for i in range(len(values)):
      if solver.BestSolutionContains(i):
          total_weight += weights[0][i]

  return computed_value, total_weight, optimality

In [ ]:
# # Initial result file
# with open('/content/Knapsack/result.csv', 'w') as f:
#   f.write('Name,Computed Value,Total Weight,Optimality\n')

In [11]:
!cat /content/Knapsack/result.csv

Name,Computed Value,Total Weight,Optimality
00Uncorrelated/n00050/R10000/s035.kp,211646,118558,Optimal
00Uncorrelated/n00100/R10000/s098.kp,414026,246938,Optimal
00Uncorrelated/n00200/R01000/s033.kp,77511,50643,Optimal
00Uncorrelated/n00500/R10000/s054.kp,2041925,1251388,Optimal
00Uncorrelated/n01000/R10000/s011.kp,4081552,2413442,Optimal
00Uncorrelated/n02000/R01000/s086.kp,796177,500746,Optimal
00Uncorrelated/n05000/R10000/s000.kp,20398463,12305918,Optimal
00Uncorrelated/n10000/R01000/s012.kp,4058343,2483579,Optimal
01WeaklyCorrelated/n00050/R10000/s083.kp,131314,122146,Optimal
01WeaklyCorrelated/n00100/R01000/s036.kp,27241,24762,Optimal
01WeaklyCorrelated/n00200/R10000/s074.kp,579182,522509,Optimal
01WeaklyCorrelated/n00500/R01000/s039.kp,131292,118451,Optimal
01WeaklyCorrelated/n01000/R10000/s073.kp,2648754,2411830,Optimal
01WeaklyCorrelated/n02000/R01000/s092.kp,543916,494716,Optimal
01WeaklyCorrelated/n05000/R01000/s086.kp,1370227,1238153,Optimal
01WeaklyCorrelated/n10000/R01000/

In [5]:
time_limit_seconds = 60

In [14]:
with open('/content/Knapsack/result.csv', 'r') as f:
  j = len(f.readlines()) - 1

# Continue solve test cases
for i in range(j,len(cases)):
  values, weights, capacities = read_case(cases[i])
  result = solve(values, weights, capacities, time_limit_seconds)
  result = [str(cases[i])[15:]] + list(map(str, result))
  f = open('/content/Knapsack/result.csv', 'a')
  f.write(','.join(result)+'\n')
  f.close()
  # break